In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 500)

In [4]:
df = pd.read_csv('data/liwc_raw_scores.csv') 
X = df.drop(['party','userid'], axis=1)
y = df['party']


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from keras.utils import to_categorical

X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=1, stratify=y)

label_encoder = LabelEncoder()
label_encoder.fit(y_train)

y_train_encoded = label_encoder.transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

y_train_category = to_categorical(y_train_encoded)
y_test_category = to_categorical(y_test_encoded)


In [6]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(units=100,activation='relu', input_dim=X.shape[1]))
model.add(Dense(units=100,activation='relu'))
model.add(Dense(units=2, activation='softmax'))
          

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

result = model.fit(X_train, y_train_category,epochs=100, shuffle=True,verbose=2)

Epoch 1/100
 - 0s - loss: 0.7895 - acc: 0.5758
Epoch 2/100
 - 0s - loss: 0.5888 - acc: 0.7083
Epoch 3/100
 - 0s - loss: 0.5052 - acc: 0.7538
Epoch 4/100
 - 0s - loss: 0.4659 - acc: 0.7803
Epoch 5/100
 - 0s - loss: 0.4276 - acc: 0.8295
Epoch 6/100
 - 0s - loss: 0.4228 - acc: 0.7992
Epoch 7/100
 - 0s - loss: 0.3882 - acc: 0.8220
Epoch 8/100
 - 0s - loss: 0.3671 - acc: 0.8371
Epoch 9/100
 - 0s - loss: 0.3507 - acc: 0.8409
Epoch 10/100
 - 0s - loss: 0.3458 - acc: 0.8485
Epoch 11/100
 - 0s - loss: 0.3272 - acc: 0.8523
Epoch 12/100
 - 0s - loss: 0.3410 - acc: 0.8447
Epoch 13/100
 - 0s - loss: 0.3296 - acc: 0.8636
Epoch 14/100
 - 0s - loss: 0.3107 - acc: 0.8561
Epoch 15/100
 - 0s - loss: 0.2936 - acc: 0.8712
Epoch 16/100
 - 0s - loss: 0.2914 - acc: 0.8561
Epoch 17/100
 - 0s - loss: 0.3002 - acc: 0.8598
Epoch 18/100
 - 0s - loss: 0.2923 - acc: 0.8902
Epoch 19/100
 - 0s - loss: 0.2831 - acc: 0.8902
Epoch 20/100
 - 0s - loss: 0.3062 - acc: 0.8598
Epoch 21/100
 - 0s - loss: 0.3386 - acc: 0.8485
E

In [7]:
model_loss, model_accuracy = model.evaluate(
    X_test, y_test_category, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

Normal Neural Network - Loss: 0.6424089352066598, Accuracy: 0.7752808915095383


In [8]:
print(result.history['acc'])
print(result.history['loss'])

[0.5757575757575758, 0.7083333333333334, 0.7537878787878788, 0.7803030303030303, 0.8295454545454546, 0.7992424242424242, 0.821969696969697, 0.8371212121212122, 0.8409090909090909, 0.8484848484848485, 0.8522727272727273, 0.8446969696969697, 0.8636363636363636, 0.8560606060606061, 0.8712121212121212, 0.8560606060606061, 0.8598484848484849, 0.8901515151515151, 0.8901515151515151, 0.8598484848484849, 0.8484848484848485, 0.8522727272727273, 0.8863636363636364, 0.8787878787878788, 0.8939393939393939, 0.9128787878787878, 0.9128787878787878, 0.9128787878787878, 0.9090909090909091, 0.9204545454545454, 0.9053030303030303, 0.8939393939393939, 0.9242424242424242, 0.9242424242424242, 0.9393939393939394, 0.9431818181818182, 0.9507575757575758, 0.9696969696969697, 0.9659090909090909, 0.9583333333333334, 0.9621212121212122, 0.9659090909090909, 0.9621212121212122, 0.9810606060606061, 0.9696969696969697, 0.9621212121212122, 0.9848484848484849, 0.9772727272727273, 0.9810606060606061, 0.9621212121212122, 

In [11]:
import json

exDict = {'model_loss': model_loss, 'model_accuracy' : model_accuracy, 
          'acc':result.history['acc'], 'loss' :result.history['loss']}

with open('data/matrix/file.txt', 'w') as file:
     file.write(json.dumps(exDict))

In [13]:
d2 = json.load(open("data/matrix/file.txt"))


dict

In [ ]:
from keras.models import load_model
model = load_model("models/raw_full.h5")
#test = np.expand_dims(X_train[0], axis=0)

y_prob = model.predict(X_train[:1])
model.predict_classes(X_train[:1])
print(f"Predicted class: {model.predict_classes(X_train[5:6])}")
y_classes = y_prob.argmax(axis=-1)
y_prob
model.

In [2]:
from keras.models import load_model
model = load_model("models/raw_full.h5")



In [ ]:
X_train[:6]


In [ ]:
X_train.iloc[0].shape

In [ ]:
predictions = model.predict_classes(X_test[:10])
prediction_labels = label_encoder.inverse_transform(predictions)
prediction_labels

print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {list(y_test[:10])}")

In [ ]:
result = model.fit(X_train, y_train_category,epochs=100, shuffle=True,verbose=2)
#history = model.fit(X, Y, validation_split=0.33, nb_epoch=150, batch_size=10, verbose=0)
print(result.history['acc'])
print(result.history['loss'])

#print(result.history.keys())


In [ ]:

rf = RandomForestClassifier()
rf.fit(X,y)

imp_feature = rf.feature_importances_
sorted_feature = np.argsort(imp_feature)
pos = np.arange(sorted_feature.shape[0]) + 0.5
plt.barh(pos, imp_feature[sorted_feature], align='center', color='darkgreen')
plt.title('Feature Importance')
plt.xlabel('Relative Feature Importance')
plt.yticks(pos, X.columns[sorted_feature])
fig = plt.gcf()
fig.set_size_inches(10,20)

In [ ]:
import matplotlib.pyplot as plt

history = model.fit(x, y, validation_split=0.25, epochs=50, batch_size=16, verbose=1)

# Plot training & validation accuracy values
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()



In [ ]:
# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
from keras.utils import plot_model
plot_model(model, to_file='model.png')